# Install

In [ ]:
# Libraries you might not have
# !python -m pip install tomotopy
# !python -m pip install sklearn

# Imports

In [ ]:
import nltk
# Things to install from nltk
nltk.download('punkt')
nltk.download('stopwords')

In [41]:
%load_ext autoreload
%autoreload 2

import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict

import dataloader
import bow
import slda
import post_classifier
import aggregate
import user_classifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Process Data

In [7]:
# Uncomment below to process data if you have not already

# POSTPATH = './Data/crowd/train/shared_task_posts.csv'
# LABELPATH = './Data/crowd/train/crowd_train.csv'
# USERPATH = './Data/crowd/train/task_C_train.posts.csv'

# users = dataloader.load_user_subset_from_train(USERPATH, subset = 1000)
    
# user_to_post, post_to_words, post_to_metadata = dataloader.load_posts(POSTPATH, user_subset = users, append_title = True)
# post_to_label = dataloader.load_classification(LABELPATH, user_to_post, post_to_words, post_to_metadata)
# filtered_data, sw_posts, sw_timestamps = dataloader.filter_posts(post_to_label, post_to_metadata)
# print(len(filtered_data))
# filtered_data = dataloader.filter_near_SW(filtered_data,post_to_metadata, sw_timestamps)
# print(len(filtered_data))

# filtered_data = dataloader.filter_stopwords(filtered_data)
# sw_posts = dataloader.filter_stopwords(sw_posts)

In [8]:
# FOLDERPATH = './Processing/crowd_processed/'
# dataloader.save_to_folder(FOLDERPATH, user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps)

# Load Process Data

In [62]:
FOLDERPATH = './Processing/crowd_processed/'
user_to_post, post_to_metadata, filtered_data, sw_posts, sw_timestamps = dataloader.load_from_folder(FOLDERPATH)

In [10]:
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'a']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'b']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'c']))
print(len([filtered_data[key] for key in filtered_data.keys() if filtered_data[key][2] == 'd']))
print(len(filtered_data))

878
203
632
2025
29200


# SLDA Model

## Train Topic Model

In [11]:
model = slda.train_slda_model_from_data(filtered_data, topics=40)

100%|██████████| 29200/29200 [00:01<00:00, 20161.34it/s]


Beginning sLDA training...
Iteration: 0	Log-likelihood: -10.155174714870453
Iteration: 100	Log-likelihood: -9.182212351170886
Iteration: 200	Log-likelihood: -9.027627637009337
Iteration: 300	Log-likelihood: -8.973904717006718
Iteration: 400	Log-likelihood: -8.942792285616337
Iteration: 500	Log-likelihood: -8.920482616658918
Iteration: 600	Log-likelihood: -8.908086957365704
Iteration: 700	Log-likelihood: -8.900066652373619
Iteration: 800	Log-likelihood: -8.890153552408961
Iteration: 900	Log-likelihood: -8.886540393409962
Finished Training


## Topic Model Outputs

In [21]:
slda_coefficients = model.get_regression_coef(0)
data = []
for k in range(model.k):
    top_words = model.get_topic_words(k, top_n=40)
    words = [word for (word, float) in top_words]
    words = ", ".join(words)
    data.append([words, slda_coefficients[k]])
    
indices = np.array(slda_coefficients).argsort()
data = np.array(data)
data = data[indices]

pd.DataFrame(data, columns=["Topic", "Suicidality Coefficient"])

,Topic,Suicidality Coefficient
0,"person, babes, big, gt, girl, f, sexy, teen, b...",-5.797159671783447
1,"url, us, min, order, person, mail, survey, gt9...",-5.00094747543335
2,"person, oc, x, free, mobile, india, app, aband...",-4.758116245269775
3,"person, music, song, video, love, album, favor...",-4.674135208129883
4,"found, killing, person, tea, pm, dropped, drag...",-4.3624749183654785
5,"person, new, us, biodynamic, american, ’, news...",-4.141739845275879
6,"live, im, webcam, sexy, person, gay, tell, wri...",-4.132311820983887
7,"person, keys, iphone, app, amp, price, 4, 2, u...",-4.103466510772705
8,"car, bike, area, city, new, ride, around, plac...",-4.044631004333496
9,"person, ’, cards, w, deck, “, ”, h, pokemon, s...",-4.016353130340576


In [22]:
# Uncomment to print example of overly negative topic
# print(data[39][0])

## sLDA Features

In [14]:
vector = slda.get_topic_vecs(model, filtered_data)

  0%|          | 0/29200 [00:00<?, ?it/s]

Getting topic distributions...


100%|██████████| 29200/29200 [27:04<00:00, 17.97it/s] 


In [68]:
#Uncomment to print example feature vector
print(vector['hw4uh'][0])

[0]


In [29]:
post_to_metadata['hw4uh']

(1307680918,
 'swtor',
 'Reddit is not Twitter.  Can we stop the accounts made specifically to content spam the subreddit?')

# Post Classifier

## Train Post Classifier

In [16]:
X = np.array([ vector[key][0] for key in vector.keys()])
y = np.array([ vector[key][1] for key in vector.keys()])
y= y.reshape(np.shape(y)[0])

In [17]:
print(np.shape(X))
print(np.shape(y))

(29200, 40)
(29200,)


In [18]:
p_clf = post_classifier.PostClassification("RbfSVM")
p_clf.train(X,y)

## Predict Post Classifier

In [19]:
y_pred = p_clf.test(X)

In [20]:
y_pred

array([1, 1, 1, ..., 0, 0, 0])

In [20]:
p_clf.get_metrics(y, y_pred)

{'accuracy': 0.7263356164383562,
 'precision': 0.17540805223068553,
 'recall': 0.7960493827160494,
 'f1': 0.2874721355327686}

In [21]:
print(sum(y_pred))
print(sum(y))

9190
2025


# User Classfier

In [79]:
for data in filtered_data:
    print(filtered_data[data])

(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a')
(37006, [], 'a

(-42907, [['trying', 'understand', 'futures', 'trading', 'works', 'cant', 'really', 'find', 'pricedwhen', 'says', 'gold', 'futures', '1600', 'mean', 'pay', '1600', 'buy', '1', 'futurearent', 'futures', 'contracts', 'bind', 'legally', 'buy', 'sell', 'given', 'commodity', 'future', 'price'], ['id', 'expect', 'contract', 'cost', 'like', '20', 'like', 'options', 'contracts'], ['anyone', 'explain']], 'None')
(-42907, [['gotten', 'habit', 'checking', 'url', 'stock', 'futures', 'bloomberg', 'market', 'opens', 'also', 'bloomberg', 'tv', 'background', 'knew', 'going', 'marketsi', 'noticed', 'thisshortly', 'market', 'open', 'theyd', 'say', 'stock', 'market', 'futures', 'higher', 'morning', 'though', 'look', 'numbers', 'website', 'like', '50', 'points', 'lower', 'yesterdays', 'close'], ['hell', 'mean', 'say', 'futures', 'higher'], ['shows', 'like', '30', 'green', 'website', 'given', 'future'], ['like', 'today', 'futures', '12240', 'market', 'open', 'market', 'closed', '12290', 'yesterday'], ['sho

(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [['get', 'really', 'distracted', 'im', 'using', 'laptop', 'reddit', 'internet', 'music', 'reddit'], ['im', 'looking', 'laptop', 'person'], ['thing']], 'None')
(-49150, [], 'None')
(-49150, [['notice', 'go', 'dress', 'socks', 'pretty', 'quickly'], ['wearing', 'would', 'see', 'new', 'hole', 'heel', 'part', 'socks'], ['way', 'prevent'], ['bonus', 'socks', 'wont', 'stay']], 'None')
(-49150, [], 'None')
(-49150, [['might', 'long', 'shot', 'maybe', 'rbooks', 'help'], ['im', 'reading', 'old', 'penguin', 'publishing', 'books', '40s50s', 'noticed', 'us', 'headquarters', 'located', 'town', 'bergenfield', 'nj'], ['grew', 'near', 'town', 'im', 'wondering', 'happened'], ['info', 'would', 'great'], ['thanks']], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [], 'None')
(-49150, [['realized', 'certain', 'teas', 'green', 'white', 'usually', 'need', 'brewed'

(-22453, [], 'None')
(-22453, [], 'None')
(-22453, [], 'None')
(-22453, [['play', 'piano', 'recently', 'ive', 'noticed', 'gotten', 'harder', 'play', 'person', 'piece', 'petite', 'chien', 'keep', 'messing', 'wondering', 'anyone', 'else', 'similar', 'experiences'], ['randomly', 'happened', 'new', 'piece', 'explanations', 'could']], 'None')
(-22453, [], 'None')
(-22453, [], 'None')
(-22453, [], 'None')
(-22453, [['tugging', 'pulling', 'shirt', 'pulling', 'pants', 'constantly', 'yeah']], 'None')
(-22453, [['old', 'keyboard', 'grew', 'hasnt', 'actually', 'used', '2', 'years'], ['larger', 'one', 'sometimes', 'like', 'go', 'back', 'play', 'old', 'one', 'remembering', 'roots'], ['dad', 'making', 'give', 'away', 'family', 'friends', '7', 'year', 'old', 'son'], ['know', 'immature', 'cant', 'help', 'feel', 'little', 'bit', 'resentful', 'like', 'im', 'giving', 'away', 'piece', 'history'], ['im', 'selfish', 'know']], 'None')
(-22453, [], 'None')
(-22453, [], 'None')
(-22453, [['okay', 'ive', 'playi

In [89]:
# change y from a, b, c, d, control to -, 1
user_to_y = defaultdict(int)
for data in tqdm.tqdm(filtered_data.keys()):
    user_to_y[filtered_data[data][0]] = (1 if filtered_data[data][2] == 'd' else 0)


100%|██████████| 2582/2582 [00:00<00:00, 27013.99it/s]


In [91]:
# user_to_y

In [43]:
post_to_uypred = defaultdict(list)

for i, post_id in enumerate(vector.keys()):
    user_id = filtered_data[post_id][0]
    post_to_uypred[post_id] = [user_id, y_pred[i]]

In [44]:
post_to_uypred

defaultdict(list,
            {'hw4uh': [8720, 1],
             'i87j9': [8720, 1],
             '2tqjks': [32730, 1],
             '2ude6n': [32730, 1],
             '2ufui5': [32730, 1],
             '2vqdhe': [32730, 0],
             '2vztle': [32730, 1],
             '2w83s8': [32730, 1],
             'd6rnd': [19994, 0],
             'd6xue': [19994, 1],
             'd9qvi': [19994, 0],
             'dbtzv': [19994, 0],
             't5iw6': [44264, 1],
             't7jrf': [44264, 1],
             'tc19h': [44264, 1],
             'tfof4': [44264, 1],
             'tfol5': [44264, 0],
             'toiq9': [44264, 1],
             'txutn': [44264, 1],
             'u1rvg': [44264, 0],
             'gexy7': [51023, 0],
             'nhfdn': [22733, 1],
             'nhfem': [22733, 1],
             'nhpy8': [22733, 0],
             'nilpl': [22733, 1],
             'nk9py': [22733, 1],
             'nlyhr': [22733, 1],
             'nsbku': [22733, 0],
             'nwznd': [227

In [57]:
user_to_post_label = aggregate.aggregate_posts(FOLDERPATH, post_to_uypred)

Saved aggreagation of user to post labels...


In [93]:
u_clf = user_classifier.UserClassification(user_to_post_label)
user_to_ypred = u_clf.argmax()

In [95]:
# user_to_y
# user_to_ypred

In [96]:
y = []
y_pred = []
for user_id in user_to_ypred:
    y.append(user_to_y[user_id])
    y_pred.append(user_to_ypred[user_id])

In [97]:
test.get_metrics(y, y_pred)

{'accuracy': 0.5745920745920746,
 'precision': 0.049738219895287955,
 'recall': 0.9047619047619048,
 'f1': 0.09429280397022331}

In [ ]:
# word2index,index2word = bow.generate_vocabulary(filtered_data)
# vector2 = bow.get_vectors_from_post_set(filtered_data, word2index)

 50%|█████     | 14732/29200 [11:47<6:17:58,  1.57s/it]  

In [ ]:
# print(word2index)

In [ ]:
# print(vector2)

In [ ]:
# y_pred